### LSTM

In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-I.csv', parse_dates=['Date'], index_col=['Date'])
data

,Open,High,Low,Close,Adj Close,Volume,ema20,ema50,rsi,macd,macd_signal,Price_Up
Date,,,,,,,,,,,,
2014-11-05,330.683014,343.368988,330.683014,339.485992,339.485992,19817200,349.705580,374.193837,41.651927,-13.723550,-12.462599,1
2014-11-06,339.458008,352.966003,338.424011,349.290009,349.290009,18797000,349.666002,373.217216,46.905741,-12.079876,-12.386054,0
2014-11-07,349.817993,352.731995,341.776001,342.415009,342.415009,16834200,348.975431,372.009287,43.919280,-11.202868,-12.149417,1
2014-11-08,342.153992,347.032013,342.153992,345.488007,345.488007,8535470,348.643295,370.969237,45.586943,-10.142946,-11.748123,1
2014-11-09,345.376007,363.626007,344.255005,363.264008,363.264008,24205600,350.035744,370.667071,54.091375,-7.778906,-10.954279,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-21,97756.195312,99507.101562,96426.523438,97224.726562,97224.726562,51765334294,99069.601344,92289.965450,48.952697,1965.864891,3012.484494,0
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,95104.937500,43147981314,98692.014312,92400.356511,44.934433,1437.643624,2697.516320,0
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,94686.242188,65239002919,98310.512205,92489.999087,44.163328,974.011455,2352.815347,1


In [4]:
features = ['Close', 'ema20', 'ema50', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target].to_numpy()
y.shape, y

((3704, 1),
 array([[1],
        [0],
        [1],
        ...,
        [1],
        [0],
        [0]], shape=(3704, 1)))

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X) # Ya lo convierte en un array de numpy 
X_scaled.shape, X_scaled

((3704, 6),
 array([[-0.99695396, -0.99756459, -0.99696886, -0.24790365, -0.16718794,
         -0.18530877],
        [-0.99676891, -0.99756539, -0.99698993, -0.1233791 , -0.16691631,
         -0.18529469],
        [-0.99689868, -0.99757928, -0.99701598, -0.19416342, -0.16677138,
         -0.18525113],
        ...,
        [ 0.78380352,  0.9735338 ,  0.98976491, -0.18837907, -0.00395957,
          0.25001379],
        [ 0.85911039,  0.97423437,  0.99499708,  0.00978761, -0.01322471,
          0.19719707],
        [ 0.8585732 ,  0.97481368,  1.        ,  0.00835724, -0.02258764,
          0.15285817]], shape=(3704, 6)))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.1, shuffle=False)

### Implementacion con Pytorch

In [7]:
# Convertimos los datos de Numpy a Tensores de Pytorch 
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape 

(torch.Size([3333, 6]),
 torch.Size([371, 6]),
 torch.Size([3333, 1]),
 torch.Size([371, 1]))

In [8]:
# Creamos Datasets y Dataloaders de Pytorch
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor) # Instanciamos la clase TendorDataSet 
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size, shuffle=False) # Instanciamos la clase DataLoader
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

print(len(train_loader))
for batch_X, batch_y in train_loader:
    print("Batch X shape:", batch_X.shape)
    print("Batch y shape:", batch_y.shape)
      # Solo el primer batch
    break


105
Batch X shape: torch.Size([32, 6])
Batch y shape: torch.Size([32, 1])


In [9]:
import torch
import torch.nn as nn 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

class MiLSTM (nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        # Definimos 1 capa LSTM y un Full Connected 
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) #Ej [8, 64, 2, (batch_size, seq_length, hidden_size)]
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Estados ocultos y celdas en 0 (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]
        x = x.unsqueeze(1)
        # Pasa la entrada por la capa LSTM 
        out, _ = self.lstm(x, (h0,c0)) #LLamo a la instancia de lstm (llamo a su metodo __call__ (no necesita nombre)) (batch_size, seq_length, hidden_size) [32, 30, 64]

        #Seleccionamos la ultima salida de la secuencia
        out = out[:,-1,:] # (batch_size , hidden_size) [32,64]

        #Pasa por la fc
        out = self.fc(out) # (batch_size, output_size) Ej [32,1]

        return out

# Parametros del modelo 
input_size = len(features) 
hidden_size = 128
num_layers  = 4
output_size = 1 # Clasificacion 

model = MiLSTM(input_size, hidden_size, num_layers, output_size).to(device)
print(model) 

cuda
MiLSTM(
  (lstm): LSTM(6, 128, num_layers=4, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


In [10]:
# Definir la función de pérdida y el optimizador
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 100 
model.train()
print(f'Entrenado en {device}')

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device) #[32, 30, 8]
        batch_y = batch_y.to(device) #[32, 1]

        outputs = model(batch_X).squeeze() # [batch_size, 1] => [batch_size] 

        loss = criterion(outputs, batch_y.squeeze())

        loss.backward()

        optimizer.step()

        optimizer.zero_grad()
    if (epoch + 1 ) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')





Entrenado en cuda
Epoch [10/100], Loss: 0.6934
Epoch [20/100], Loss: 0.7069
Epoch [30/100], Loss: 0.7077
Epoch [40/100], Loss: 0.7076
Epoch [50/100], Loss: 0.7060
Epoch [60/100], Loss: 0.7034
Epoch [70/100], Loss: 0.6962
Epoch [80/100], Loss: 0.6776
Epoch [90/100], Loss: 0.6820
Epoch [100/100], Loss: 0.6581


In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

model.eval()  # Modo de evaluación
y_true = []  # Etiquetas reales
y_pred = []  # Predicciones binarias
y_probs = []  # Probabilidades predichas

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X).squeeze()   # Logits
        probs = torch.sigmoid(outputs)       # Probabilidades
        predicted = torch.round(probs)       # Predicciones binarias (0 o 1)

        # Guardar los valores reales y predichos
        y_true.extend(batch_y.squeeze().cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
        y_probs.extend(probs.cpu().numpy())

# Convertir listas a numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

# Calcular métricas
accuracy = np.mean(y_pred == y_true) * 100
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_probs)

# Mostrar resultados
print(f'Accuracy: {accuracy:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')


Accuracy: 52.56%
Precision: 0.59
Recall: 0.28
F1-Score: 0.38
ROC-AUC: 0.51


In [13]:
import joblib

# Guardar el modelo y el scaler
torch.save(model, 'model.pth')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']